In [1]:
import sys; sys.argv=['python']


import network
from utils.argparse_utils import add_bool_arg

import argparse

parser = argparse.ArgumentParser()

# ------------------ directories -----------------
parser.add_argument("--train_list", type=str, default=None)
parser.add_argument("--val_list", type=str, default=None)
parser.add_argument("--test_list", type=str, default=None)
parser.add_argument(
    "--root_dir", type=str, default=None, help="Path to data directory"
)
parser.add_argument("--out_dir", type=str, default=None)
parser.add_argument("--model_path", type=str, default=None)
parser.add_argument("--continue_from", type=str, default=None)
parser.add_argument("--out_name", type=str, default=None)

# ------------------ architecture and training -----------------
parser.add_argument("--lr", type=float, default=1e-04)
parser.add_argument("--batch_size", type=int, default=128)
parser.add_argument("--epochs", type=int, default=10000, help="max num epochs")
parser.add_argument("--arch", type=str, default="resnet")
parser.add_argument("--cpu", action="store_true")
parser.add_argument("--input_dim", type=int, default=6)
parser.add_argument("--output_dim", type=int, default=3)

# ------------------ commons -----------------
parser.add_argument(
    "--mode", type=str, default="train", choices=["train", "test", "eval"]
)
parser.add_argument(
    "--imu_freq", type=float, default=200.0, help="imu_base_freq is a multiple"
)
parser.add_argument("--imu_base_freq", type=float, default=1000.0)

# ----- perturbation -----
add_bool_arg(parser, "do_bias_shift", default=True)
parser.add_argument("--accel_bias_range", type=float, default=0.2)  # 5e-2
parser.add_argument("--gyro_bias_range", type=float, default=0.05)  # 1e-3

add_bool_arg(parser, "perturb_gravity", default=True)
parser.add_argument(
    "--perturb_gravity_theta_range", type=float, default=5.0
)  # degrees

# ----- window size and inference freq -----
parser.add_argument("--past_time", type=float, default=0.0)  # s
parser.add_argument("--window_time", type=float, default=1.0)  # s
parser.add_argument("--future_time", type=float, default=0.0)  # s

# ----- for sampling in training / stepping in testing -----
parser.add_argument("--sample_freq", type=float, default=20.0)  # hz

# ----- plotting and evaluation -----
add_bool_arg(parser, "save_plot", default=False)
parser.add_argument("--rpe_window", type=float, default="2.0")  # s

args = parser.parse_args()


In [2]:
import glob, os
from torch.utils.data import DataLoader
import json
import os
import signal
import sys
import time
from functools import partial
from os import path as osp

import numpy as np
import torch
from network.losses import get_loss
from network.model_factory import get_model
from torch.utils.data import DataLoader

from dataloader.dataset_subt import SubtSequneceDataset


args.train_list = []
args.test_list = []
data_list = []
args.root_dir = "/media/yuheng/74123e5f-dc58-4223-9aa8-60f21c431fb1/data/subt_IMU"
paths = glob.glob(args.root_dir+"/*")
for p in paths:
    if osp.isdir(p):
        data_list.append(p.split("/")[-1])
args.test_list = data_list[:1]
args.train_list = data_list[1:]

data_window_config = dict(
    [
        ("past_data_size", int(args.past_time * args.imu_freq)),
        ("window_size", int(args.window_time * args.imu_freq)),
        ("future_data_size", int(args.future_time * args.imu_freq)),
        ("step_size", int(args.imu_freq / args.sample_freq)),
    ]
)
net_config = {
    "in_dim": (
        data_window_config["past_data_size"]
        + data_window_config["window_size"]
        + data_window_config["future_data_size"]
    )
    // 32
    + 1
}


In [23]:
data_window_config

{'past_data_size': 0,
 'window_size': 200,
 'future_data_size': 0,
 'step_size': 10}

In [6]:
train_dataset = SubtSequneceDataset(
        args.root_dir, args.train_list, args, data_window_config, mode="train")
train_loader = DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=True
    )

test_dataset = SubtSequneceDataset(
        args.root_dir, args.train_list, args, data_window_config, mode="test")
test_loader = DataLoader(
        test_dataset, batch_size=args.batch_size, shuffle=True
    )

device = torch.device(
    "cuda:0" if torch.cuda.is_available() and not args.cpu else "cpu"
)
network = get_model(args.arch, net_config, args.input_dim, args.output_dim).to(
    device
)

In [7]:
from network.train import do_train, get_inference
optimizer = torch.optim.Adam(network.parameters(), args.lr)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True, eps=1e-12
)


In [24]:
epoch = 1
for i in range(100):
    train_attr_dict = do_train(network, train_loader, device, epoch, optimizer)
    val_attr_dict = get_inference(network, test_loader, device, epoch)
    train_loss = train_attr_dict['losses'].mean()
    test_loss = val_attr_dict['losses'].mean()
    print("train %f, test %f"%(train_loss, test_loss))

train 1.133689, test 0.810610
train 1.171009, test 0.725676
train 1.082600, test 0.708741
train 1.088306, test 0.730195
train 0.921712, test 0.708151
train 1.034921, test 0.693949
train 0.980930, test 0.667365
train 1.061090, test 0.671228
train 0.988643, test 0.669478
train 0.962827, test 0.651188
train 0.993132, test 0.665350
train 1.005542, test 0.654283
train 0.974072, test 0.614874
train 0.932103, test 0.605898
train 0.942258, test 0.610768
train 0.875037, test 0.671589
train 0.894006, test 0.794047
train 0.861211, test 0.913793
train 0.735889, test 0.831455
train 0.846869, test 0.738356
train 0.836337, test 0.572841
train 0.834580, test 0.476665
train 0.824875, test 0.491001
train 0.943350, test 0.498550
train 0.779857, test 0.480328
train 0.762601, test 0.480787
train 0.749256, test 0.518363
train 0.753861, test 0.496877
train 0.833359, test 0.445269
train 0.703393, test 0.393174
train 0.783937, test 0.397077
train 0.780241, test 0.409044
train 0.777862, test 0.419267
train 0.68